In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
import cv2
import xml.etree.ElementTree as ET 
import numpy as np
import sys
from getBoundingBoxes import *
from corner_detector import *
from getBoxFeatures import *
from anms import *
from utils import *
from estimateAllTranslation import *
from estimateFeatureTranslation import *
from applyBoxTransform import *

In [ ]:
cap = cv2.VideoCapture('vids/Easy.mp4')
ret,firstFrame = cap.read()
boxes = getBoundingBoxes()
gray = cv2.cvtColor(firstFrame,cv2.COLOR_BGR2GRAY)

boxesData = []

for box in boxes:
    boximg = gray[box[1]:box[3],box[0]:box[2]]
    x,y = getBoxFeatures(boximg,box,100)
    boxData = {
        'coords': box,
        '_x': x,
        '_y': y
    }
    boxesData.append(boxData)

currentFrame = firstFrame
_,nextFrame = cap.read()
gray = cv2.cvtColor(currentFrame,cv2.COLOR_BGR2GRAY)


In [ ]:
# Imag, Ix, Iy,Iori=findDerivatives(gray)
# plt.imshow(Imag)
_x = boxesData[0]['_x']
_y = boxesData[0]['_y']
X,Y=estimateAllTranslation(_x,_y,currentFrame,nextFrame)


print(X-_x)
print(Y-_y)

In [ ]:
plt.imshow(gray)
fig = plt.gcf()
ax1 = fig.add_subplot(111)
ax1.scatter(_x,_y,c='r',s=1)

In [ ]:
plt.imshow(cv2.cvtColor(nextFrame,cv2.COLOR_BGR2GRAY))
fig = plt.gcf()
ax1 = fig.add_subplot(111)
ax1.scatter(X,Y,c='r',s=1)


In [ ]:
h = est_homography(_x,_y,X,Y)

In [ ]:
pt = np.array([_x[0],_y[0],1]).T
newpt = h@pt
newpt = newpt/newpt[2]
print(newpt)
print(np.array([X[0],Y[0],1]))

In [ ]:
_x.shape

In [ ]:
n = _x.shape[0]
stk = np.vstack((_x,_y,np.ones(n)))

newstk = np.vstack((X,Y,np.ones(n)))

In [ ]:
res=h@stk
res=res/res[2,:]

In [ ]:
bdata=boxesData[0]['coords']
X,Y,_box = applyBoxTransform(_x,_y,X,Y,bdata)

In [ ]:
_box

In [ ]:
bdata

In [ ]:
X-_x